# Concurrency Error Handling - Backend-Agnostic Utilities

The `concurrency._errors` module provides utilities for handling cancellation and exceptions in async code that works across asyncio and trio backends.

**Core Features:**
- **Backend Detection**: Auto-detect cancellation exception class (asyncio.CancelledError vs trio.Cancelled)
- **Cancellation Testing**: Check if exceptions are cancellations
- **Shielding**: Protect critical code from cancellation
- **Exception Groups**: Split and filter cancellations from other errors (Python 3.11+)
- **Clean Error Handling**: Separate cancellation from actual failures

In [1]:
import asyncio

import anyio

from lionherd_core.libs.concurrency import (
    get_cancelled_exc_class,
    is_cancelled,
    non_cancel_subgroup,
    shield,
    sleep,
)

# Note: split_cancellation not in public API, import from private module
from lionherd_core.libs.concurrency._errors import split_cancellation

## 1. Backend-Native Cancellation Detection

Different async backends use different cancellation exceptions. These utilities detect the correct class automatically.

In [2]:
# Get the cancellation exception class for current backend
cancel_class = get_cancelled_exc_class()
print(f"Backend cancellation class: {cancel_class.__name__}")
print(f"Module: {cancel_class.__module__}")

# In asyncio backend, this will be asyncio.CancelledError
# In trio backend, this would be trio.Cancelled

Backend cancellation class: CancelledError
Module: asyncio.exceptions


## 2. Testing for Cancellation

Use `is_cancelled()` to distinguish cancellation from other exceptions.

In [3]:
# Create different exception types
cancel_exc = asyncio.CancelledError("Task cancelled")
value_exc = ValueError("Invalid input")
runtime_exc = RuntimeError("Something went wrong")

print(f"CancelledError is cancellation: {is_cancelled(cancel_exc)}")
print(f"ValueError is cancellation: {is_cancelled(value_exc)}")
print(f"RuntimeError is cancellation: {is_cancelled(runtime_exc)}")

CancelledError is cancellation: True
ValueError is cancellation: False
RuntimeError is cancellation: False


In [4]:
# Practical use: distinguish failure types
async def robust_operation():
    """Handle cancellation differently from errors."""
    try:
        await sleep(1)
        result = 42 / 0  # Will raise ZeroDivisionError
        return result
    except Exception as e:
        if is_cancelled(e):
            print("Operation was cancelled - cleanup gracefully")
            raise  # Re-raise to propagate cancellation
        else:
            print(f"Operation failed: {type(e).__name__}: {e}")
            # Handle error, maybe retry or return default
            return None


# Test with actual error
result = await robust_operation()
print(f"Result: {result}")

Operation failed: ZeroDivisionError: division by zero
Result: None


## 3. Shielding from Cancellation

Use `shield()` to protect critical operations from being cancelled.

In [5]:
# Critical operation that must complete
async def save_to_database(data):
    """Simulate database write."""
    await sleep(0.1)
    print(f"Saved to database: {data}")
    return True


async def process_with_cleanup(value):
    """Process data with guaranteed cleanup."""
    print(f"Processing {value}...")
    await sleep(0.05)

    # Even if this function gets cancelled, cleanup will complete
    await shield(save_to_database, f"processed_{value}")
    print("Cleanup guaranteed!")


# Test normal execution
await process_with_cleanup(42)

Processing 42...
Saved to database: processed_42
Cleanup guaranteed!


In [6]:
# Test with cancellation
async def cancelled_scenario():
    """Simulate cancellation during processing."""
    with anyio.CancelScope() as scope:
        async with anyio.create_task_group() as tg:
            tg.start_soon(process_with_cleanup, 99)  # Callable, not coroutine
            await sleep(0.01)  # Let it start
            scope.cancel()  # Cancel the scope

    print("Task was cancelled, but shielded operation completed")


await cancelled_scenario()

Processing 99...
Task was cancelled, but shielded operation completed


## 4. Exception Group Splitting (Python 3.11+)

When running multiple tasks concurrently, separate cancellations from real errors using `split_cancellation()`.

In [7]:
# Create mixed exception group
exceptions = [
    asyncio.CancelledError("Task 1 cancelled"),
    ValueError("Invalid data"),
    asyncio.CancelledError("Task 3 cancelled"),
    RuntimeError("Connection failed"),
]

exc_group = BaseExceptionGroup("Multiple failures", exceptions)
print(f"Original group has {len(exc_group.exceptions)} exceptions")

# Split into cancellations and errors
cancel_group, error_group = split_cancellation(exc_group)

print(f"\nCancellation group: {cancel_group}")
if cancel_group:
    print(f"  - {len(cancel_group.exceptions)} cancellations")
    for exc in cancel_group.exceptions:
        print(f"    • {exc}")

print(f"\nError group: {error_group}")
if error_group:
    print(f"  - {len(error_group.exceptions)} actual errors")
    for exc in error_group.exceptions:
        print(f"    • {type(exc).__name__}: {exc}")

Original group has 4 exceptions

Cancellation group: Multiple failures (2 sub-exceptions)
  - 2 cancellations
    • Task 1 cancelled
    • Task 3 cancelled

Error group: Multiple failures (2 sub-exceptions)
  - 2 actual errors
    • ValueError: Invalid data
    • RuntimeError: Connection failed


## 5. Filtering Non-Cancellation Errors

Use `non_cancel_subgroup()` to get only the actual errors, ignoring cancellations.

In [8]:
# Same exception group
exc_group = BaseExceptionGroup(
    "Multiple failures",
    [
        asyncio.CancelledError("Cancelled"),
        ValueError("Bad value"),
        asyncio.CancelledError("Also cancelled"),
        TypeError("Wrong type"),
    ],
)

# Get only real errors
errors_only = non_cancel_subgroup(exc_group)

if errors_only:
    print(f"Real errors to handle: {len(errors_only.exceptions)}")
    for exc in errors_only.exceptions:
        print(f"  • {type(exc).__name__}: {exc}")
else:
    print("No real errors - all were cancellations")

Real errors to handle: 2
  • ValueError: Bad value
  • TypeError: Wrong type


In [9]:
# All cancellations - returns None
all_cancelled = BaseExceptionGroup(
    "All cancelled",
    [
        asyncio.CancelledError("Task 1"),
        asyncio.CancelledError("Task 2"),
    ],
)

errors_only = non_cancel_subgroup(all_cancelled)
print(f"Result when all cancelled: {errors_only}")
print("✓ Returns None when no real errors")

Result when all cancelled: None
✓ Returns None when no real errors


## 6. Real-World Example: Task Runner with Error Handling

Combine these utilities to build robust concurrent operations.

In [10]:
async def flaky_task(task_id: int, should_fail: bool = False):
    """Simulate a task that might fail or be cancelled."""
    await sleep(0.05)

    if should_fail:
        raise ValueError(f"Task {task_id} failed")

    return f"Result {task_id}"


async def run_tasks_with_cleanup(task_configs):
    """Run multiple tasks with proper error handling and cleanup."""
    results = []
    errors = []

    async def task_wrapper(task_id, should_fail):
        """Wrapper that captures results or errors."""
        try:
            result = await flaky_task(task_id, should_fail)
            results.append(result)
        except Exception as e:
            errors.append(e)

    # Start all tasks in TaskGroup
    async with anyio.create_task_group() as tg:
        for task_id, should_fail in task_configs:
            # Pass callable and arguments separately
            tg.start_soon(task_wrapper, task_id, should_fail)

    # After TaskGroup exits, all tasks are complete

    # Analyze errors
    if errors:
        exc_group = BaseExceptionGroup("Task failures", errors)

        # Separate cancellations from real errors
        real_errors = non_cancel_subgroup(exc_group)

        if real_errors:
            print(f"⚠️  {len(real_errors.exceptions)} tasks failed:")
            for exc in real_errors.exceptions:
                print(f"   • {exc}")

        cancel_group, _ = split_cancellation(exc_group)
        if cancel_group:
            print(f"ℹ️  {len(cancel_group.exceptions)} tasks cancelled")  # noqa: RUF001

    # Always cleanup (shielded from cancellation)
    await shield(save_to_database, f"completed_{len(results)}_of_{len(task_configs)}")

    return results


# Test with mixed success/failure
configs = [
    (1, False),  # Success
    (2, True),  # Fail
    (3, False),  # Success
    (4, True),  # Fail
]

results = await run_tasks_with_cleanup(configs)
print(f"\n✓ Completed {len(results)} successful tasks")
print(f"Results: {results}")

⚠️  2 tasks failed:
   • Task 2 failed
   • Task 4 failed
Saved to database: completed_2_of_4

✓ Completed 2 successful tasks
Results: ['Result 1', 'Result 3']


## 7. Pattern: Graceful Shutdown

Shield cleanup operations during application shutdown.

In [11]:
class Service:
    """Example service with guaranteed cleanup."""

    def __init__(self, name: str):
        self.name = name
        self.running = False

    async def start(self):
        self.running = True
        print(f"✓ {self.name} started")

    async def stop(self):
        """Critical cleanup - must complete."""
        await sleep(0.05)  # Simulate cleanup work
        self.running = False
        print(f"✓ {self.name} stopped cleanly")

    async def run(self):
        """Main service loop with guaranteed cleanup."""
        await self.start()

        try:
            # Simulate work
            while True:
                await sleep(0.1)
        except Exception as e:
            if is_cancelled(e):
                print(f"ℹ️  {self.name} received cancellation")  # noqa: RUF001
            else:
                print(f"⚠️  {self.name} error: {e}")
            raise
        finally:
            # Cleanup is shielded - will complete even if cancelled
            await shield(self.stop)


# Test graceful shutdown
async def test_graceful_shutdown():
    service = Service("DatabasePool")

    with anyio.CancelScope() as scope:
        async with anyio.create_task_group() as tg:
            tg.start_soon(service.run)  # Pass callable, not coroutine
            await sleep(0.15)  # Let it run
            scope.cancel()  # Initiate shutdown

    print("✓ Service shutdown complete")
    print(f"Final state - running: {service.running}")


await test_graceful_shutdown()

✓ DatabasePool started
✓ DatabasePool stopped cleanly
✓ Service shutdown complete
Final state - running: False


## Summary Checklist

**Concurrency Error Handling Essentials:**
- ✅ `get_cancelled_exc_class()` - Auto-detect backend cancellation exception
- ✅ `is_cancelled(exc)` - Test if exception is cancellation
- ✅ `shield(func, *args, **kwargs)` - Protect critical code from cancellation
- ✅ `split_cancellation(eg)` - Separate cancellations from errors in exception groups
- ✅ `non_cancel_subgroup(eg)` - Filter out cancellations, keep only real errors
- ✅ Backend-agnostic (works with asyncio and trio)
- ✅ Python 3.11+ ExceptionGroup support
- ✅ Preserves exception structure, tracebacks, and metadata

**Best Practices:**
- Shield cleanup operations (database commits, file closes, resource releases)
- Distinguish cancellation from failure in error handling
- Use `non_cancel_subgroup()` to focus on actionable errors
- Always re-raise cancellation after cleanup
- Combine with `TaskGroup` for robust concurrent operations

**Next Steps:**
- See `concurrency.throttle` for rate limiting
- See `concurrency.gather` for parallel task execution
- See `Event` for async event handling patterns